In [68]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta
import math

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [69]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [70]:
num_cars = 100
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.collect().head())

shape: (5, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00        ┆ 23.160743 ┆ -115.938862 ┆ engine_check  │
│ car_0  ┆ 2023-01-01T00:28:07.958717 ┆ 23.160743 ┆ -115.938862 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:02:26.435841 ┆ 23.160743 ┆ -115.938862 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:13:04.851675 ┆ 23.160743 ┆ -115.938862 ┆ engine_check  │
│ car_0  ┆ 2023-01-01T00:22:41.305753 ┆ 23.160743 ┆ -115.938862 ┆ engine_check  │
└────────┴────────────────────────────┴───────────┴─────────────┴───────────────┘


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [71]:
data = data.with_columns(pl.col("timestamp").cast(pl.Datetime))
print(data.collect())

shape: (633_100, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ datetime[μs]               ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01 00:00:00        ┆ 23.160743 ┆ -115.938862 ┆ engine_check  │
│ car_0  ┆ 2023-01-01 00:28:07.958717 ┆ 23.160743 ┆ -115.938862 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:02:26.435841 ┆ 23.160743 ┆ -115.938862 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01 00:13:04.851675 ┆ 23.160743 ┆ -115.938862 ┆ engine_check  │
│ car_0  ┆ 2023-01-01 00:22:41.305753 ┆ 23.160743 ┆ -115.938862 ┆ engine_check  │
│ …      ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_99 ┆ 2023-01-07 23:39:40.902173 ┆ 29.574764 ┆ -109.55883  ┆ null        

### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [72]:
data = data.sort('car_id', 'timestamp')
data = data.with_columns([
    pl.col("timestamp").cast(pl.Datetime),
    pl.col("timestamp").cast(pl.Datetime).diff().over(pl.col("car_id").rle_id()).alias("notification_time")
])

print(data.collect())

shape: (633_100, 6)
┌────────┬────────────────────────────┬───────────┬─────────────┬──────────────┬───────────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification ┆ notification_time │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---          ┆ ---               │
│ str    ┆ datetime[μs]               ┆ f64       ┆ f64         ┆ str          ┆ duration[μs]      │
╞════════╪════════════════════════════╪═══════════╪═════════════╪══════════════╪═══════════════════╡
│ car_0  ┆ 2023-01-01 00:00:00        ┆ 23.15476  ┆ -115.946026 ┆ low_fuel     ┆ null              │
│ car_0  ┆ 2023-01-01 00:00:00        ┆ 23.160743 ┆ -115.938862 ┆ engine_check ┆ 0µs               │
│ car_0  ┆ 2023-01-01 00:00:03.828298 ┆ 23.15476  ┆ -115.946026 ┆ engine_check ┆ 3s 828298µs       │
│ car_0  ┆ 2023-01-01 00:00:07.526891 ┆ 23.15476  ┆ -115.946026 ┆ null         ┆ 3s 698593µs       │
│ car_0  ┆ 2023-01-01 00:00:11.628824 ┆ 23.15476  ┆ -115.946026 ┆ null 

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [73]:
data = data.with_columns([
    pl.col("latitude").diff().over("car_id").alias("delta_latitude"),
    pl.col("longitude").diff().over("car_id").alias("delta_longitude"),
])
print(data.collect())

shape: (633_100, 8)
┌────────┬────────────┬───────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ car_id ┆ timestamp  ┆ latitude  ┆ longitude  ┆ notificati ┆ notificati ┆ delta_lati ┆ delta_long │
│ ---    ┆ ---        ┆ ---       ┆ ---        ┆ on         ┆ on_time    ┆ tude       ┆ itude      │
│ str    ┆ datetime[μ ┆ f64       ┆ f64        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│        ┆ s]         ┆           ┆            ┆ str        ┆ duration[μ ┆ f64        ┆ f64        │
│        ┆            ┆           ┆            ┆            ┆ s]         ┆            ┆            │
╞════════╪════════════╪═══════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ car_0  ┆ 2023-01-01 ┆ 23.15476  ┆ -115.94602 ┆ low_fuel   ┆ null       ┆ null       ┆ null       │
│        ┆ 00:00:00   ┆           ┆ 6          ┆            ┆            ┆            ┆            │
│ car_0  ┆ 2023-01-01 ┆ 23.160743 ┆ -115.93886 ┆ engine_che ┆ 0µs      

In [74]:
def harvesine(groups):
    earth_radius = 6371.0

    delta_lat = groups[0].to_numpy()
    delta_lon = groups[1].to_numpy()

    delta_lat = [math.radians(x) for x in delta_lat]
    delta_lon = [math.radians(x) for x in delta_lon]

    a = [(math.sin(lat / 2))**2 + math.cos(lat) * (math.sin(lon / 2))**2 for lat, lon in zip(delta_lat, delta_lon)]
    c = [2 * math.atan2(math.sqrt(x), math.sqrt(1 - x)) for x in a]
    distance = [earth_radius * x for x in c]

    return pl.Series(distance)


In [75]:
data = data.with_columns([pl.map_groups(["delta_latitude", "delta_longitude"], harvesine).alias("distance")])
print(data.collect())

shape: (633_100, 9)
┌────────┬────────────┬───────────┬────────────┬───┬────────────┬───────────┬───────────┬──────────┐
│ car_id ┆ timestamp  ┆ latitude  ┆ longitude  ┆ … ┆ notificati ┆ delta_lat ┆ delta_lon ┆ distance │
│ ---    ┆ ---        ┆ ---       ┆ ---        ┆   ┆ on_time    ┆ itude     ┆ gitude    ┆ ---      │
│ str    ┆ datetime[μ ┆ f64       ┆ f64        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ f64      │
│        ┆ s]         ┆           ┆            ┆   ┆ duration[μ ┆ f64       ┆ f64       ┆          │
│        ┆            ┆           ┆            ┆   ┆ s]         ┆           ┆           ┆          │
╞════════╪════════════╪═══════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪══════════╡
│ car_0  ┆ 2023-01-01 ┆ 23.15476  ┆ -115.94602 ┆ … ┆ null       ┆ null      ┆ null      ┆ NaN      │
│        ┆ 00:00:00   ┆           ┆ 6          ┆   ┆            ┆           ┆           ┆          │
│ car_0  ┆ 2023-01-01 ┆ 23.160743 ┆ -115.93886 ┆ … ┆ 0µs        ┆ 0.005

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [76]:
data = data.with_columns(pl.col('timestamp').dt.truncate('2h').alias('interval'))


In [77]:
resultado = data.group_by("car_id", "interval", maintain_order=True).agg([
    pl.col("notification_time").mean().alias("notification_time_promedio"),
    pl.col("notification_time").min().alias("notification_time_minimo"),
    pl.col("notification_time").max().alias("notification_time_maximo"),
    pl.col("notification_time").std().alias("desviacion_notification_time"),
    pl.col("distance").mean().alias("distance_promedio"),
    pl.col("distance").min().alias("distance_minimo"),
    pl.col("distance").max().alias("distance_maximo"),
    pl.col("distance").std().alias("desviacion_distance")
])

print(resultado.collect())

shape: (8_444, 10)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ interval   ┆ notificati ┆ notificat ┆ … ┆ distance_ ┆ distance_ ┆ distance_ ┆ desviacio │
│ ---    ┆ ---        ┆ on_time_pr ┆ ion_time_ ┆   ┆ promedio  ┆ minimo    ┆ maximo    ┆ n_distanc │
│ str    ┆ datetime[μ ┆ omedio     ┆ minimo    ┆   ┆ ---       ┆ ---       ┆ ---       ┆ e         │
│        ┆ s]         ┆ ---        ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ ---       │
│        ┆            ┆ duration[μ ┆ duration[ ┆   ┆           ┆           ┆           ┆ f64       │
│        ┆            ┆ s]         ┆ μs]       ┆   ┆           ┆           ┆           ┆           │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2023-01-01 ┆ 25s        ┆ 0µs       ┆ … ┆ NaN       ┆ 0.0       ┆ 1.13609   ┆ NaN       │
│        ┆ 00:00:00   ┆ 983105µs   ┆           ┆   ┆           ┆        